In [1]:
# read saved results (tables) of old runs
# combine multiple runs
# make significance plots

import sys
import pandas as pd
import numpy as np
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
import pandas as pd
import core_tools as ct
import core_calib as cal
from scipy import stats


In [2]:
main_path = './Data manipulation/results'
# exp_names = ["1692118038_T10", "1692121657_T50", "1692130088_T100", "1692161204_O50", "1692172225_O100"] # , "1692107939_Z8T0"
# exp_names = ["1692351821_RFseed"]
exp_names = ["1698037233_Paper_v2_Real30_OOB_False", "1698000930_Paper_v2_Real30_OOB_true"]
exp_comp = "_OOB"

In [3]:
# to combine Calib and OOB runs in one CD plot
tables_all = {}
for exp in exp_names:
    path = f"{main_path}/{exp}"
    dict_data, tables, params = ct.read_results(path)
    if len(tables_all) == 0:
        tables_all = tables
    else: # combine tables
        for table_key in tables.keys():
            col_names = np.array((tables[table_key].columns))
            col_names += exp_comp
            tables[table_key].columns = col_names
            # print(tables[table_key])
            # print(f"key {table_key} col_names", col_names)
            tables_all[table_key] = pd.concat([tables_all[table_key], tables[table_key]], axis=1)
            # print(tables_all[table_key])

# tables_all = ct.add_rank_mean(tables_all)
tables_all["brier"]

# ct.res_statistics(tables_all, params["metrics"], ".")

,RF_d,RF_opt,RF_large,Platt,ISO,Beta,CRF,VA,tlr,Rank,...,RF_opt_OOB,RF_large_OOB,Platt_OOB,ISO_OOB,Beta_OOB,CRF_OOB,VA_OOB,tlr_OOB,Rank_OOB,Elkan_OOB
Data,,,,,,,,,,,,,,,,,,,,,
cm1,0.087733,0.084365,8.439491e-02,0.087454,0.090743,8.716595e-02,8.631986e-02,0.091690,0.097934,0.091830,...,0.084365,8.415170e-02,0.085394,0.084884,8.399451e-02,0.084415,8.492045e-02,0.098259,0.099345,0.188567
datatrieve,0.082625,0.082833,8.103581e-02,0.092124,0.102327,1.217710e-01,8.341386e-02,0.101697,0.084718,0.080404,...,0.082833,8.202800e-02,0.080492,0.083574,8.170308e-02,0.084209,8.381957e-02,0.084638,0.089368,0.182819
kc1_class_level_defectiveornot,0.170711,0.170272,1.700769e-01,0.175447,0.206486,1.987379e-01,1.791619e-01,0.179129,0.185901,0.222564,...,0.170272,1.698602e-01,0.170810,0.177351,1.719113e-01,0.171548,1.752726e-01,0.182399,0.254978,0.259204
kc1,0.103789,0.106387,1.059902e-01,0.108332,0.109994,1.072053e-01,1.069721e-01,0.108401,0.127594,0.113276,...,0.106387,1.059076e-01,0.107345,0.107329,1.067220e-01,0.106379,1.072062e-01,0.127505,0.117140,0.223429
kc2,0.120842,0.114107,1.145694e-01,0.118206,0.122513,1.192941e-01,1.171598e-01,0.117843,0.129164,0.121790,...,0.114107,1.141509e-01,0.115462,0.115210,1.177497e-01,0.114545,1.147930e-01,0.128483,0.128280,0.217724
kc3,0.075324,0.073344,7.261353e-02,0.076794,0.082990,7.790112e-02,7.478821e-02,0.078579,0.082969,0.081862,...,0.073344,7.297966e-02,0.074632,0.073751,7.515467e-02,0.073704,7.354316e-02,0.082530,0.088458,0.157051
pc1,0.050539,0.051281,5.031371e-02,0.053812,0.057059,5.420718e-02,5.223557e-02,0.054616,0.059435,0.056419,...,0.051281,5.080691e-02,0.052023,0.052186,5.227254e-02,0.051409,5.179743e-02,0.061205,0.060952,0.114620
spect,0.131644,0.120972,1.201443e-01,0.126163,0.135456,1.279401e-01,1.197782e-01,0.127013,0.131987,0.150884,...,0.120972,1.201829e-01,0.120935,0.123970,1.216759e-01,0.121219,1.233240e-01,0.131104,0.168109,0.168686
spectf,0.127862,0.124621,1.232139e-01,0.131729,0.145698,1.368705e-01,1.317765e-01,0.132820,0.139655,0.149772,...,0.124621,1.223701e-01,0.127146,0.127380,1.257749e-01,0.125408,1.263861e-01,0.136840,0.172409,0.172415


In [4]:

def stat_test_binary(test_table, method, metric, log=False):

    # Create two sample arrays
    Normal = np.array(test_table.iloc[:, 0])
    Modified = np.array(test_table.iloc[:, 1])

    result = Normal - Modified # for loss comparison
    # result = Modified - Normal # for acc comparison
    if metric == "acc":
        result[result > 0] = 0
        result[result < 0] = 1
    else:
        result[result > 0] = 1
        result[result < 0] = 0
    n_trials = len(result) 
    observed_successes = int(sum(result))

    if log:
        print(f"{metric} {method} observed_successes", observed_successes)

    result = stats.binomtest(observed_successes, n=n_trials, p=0.5, alternative='less')
    p_value = result.pvalue

    if log:
        print("p_value", p_value)

    alpha = 0.05

    if p_value < alpha:
        if log:
            print("Reject the null hypothesis. The observed frequency is significantly different from the expected frequency.")
        return 1
    else:
        if log:
            print("Fail to reject the null hypothesis. There is no significant difference.")
        return 0

def stat_test_ttest(test_table, method, metric, log=False):

    if metric == "acc":
        statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='less')
    else:
        statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='greater')


    if log:
        print("p_value", p_value)
    alpha = 0.05

    if p_value < alpha:
        if log:
            # print("Reject the null hypothesis. The observed frequency is significantly different from the expected frequency.")
            print("The change creates significant improvment")
        return '\ding{51}'
    else:
        if metric == "acc":
            statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='greater')
        else:
            statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='less')

        if p_value < alpha:
            if log:
                print("The change creates significant decline.")
            return '\ding{55}'
        else:
            if log:    
                print("Fail to reject the null hypothesis. There is no significant difference.")
        return "  "



In [5]:
calib_methods = [   "RF_d", "RF_opt", "RF_large",
                    "Platt", "ISO", "Beta", "CRF", "VA",
                    "tlr", "Rank", #"Elkan",                      
                ]

metrics =  ["acc", "brier", "logloss", "ece"]

res_table = []
for method in calib_methods:
    res_array = [method]
    for metric in metrics:
        test_table = tables_all[metric][[method, method + exp_comp]]

        if method == "RF_opt" and metric == "brier":
            print(test_table)

        res_array.append(stat_test_ttest(test_table, method, metric))
    res_table.append(res_array)

res_table = np.array(res_table)

res_table = pd.DataFrame(res_table, columns=["calib_methods"]+ metrics)
res_table.set_index('calib_methods', inplace=True)
res_table.to_latex(f"./OOB_stat_test.txt")
res_table

                                  RF_opt  RF_opt_OOB
Data                                                
cm1                             0.084365    0.084365
datatrieve                      0.082833    0.082833
kc1_class_level_defectiveornot  0.170272    0.170272
kc1                             0.106387    0.106387
kc2                             0.114107    0.114107
kc3                             0.073344    0.073344
pc1                             0.051281    0.051281
spect                           0.120972    0.120972
spectf                          0.124621    0.124621
vertebral                       0.109777    0.109777
wilt                            0.011505    0.011505
parkinsons                      0.093948    0.093948
heart                           0.128106    0.128106
wdbc                            0.029972    0.029972
bank                            0.005698    0.005698
ionosphere                      0.054105    0.054105
HRCompetencyScores              0.063633    0.

/var/folders/gm/nlbfzh25075c0jwn9khghrfw0000gn/T/ipykernel_65842/1496434187.py:24: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  res_table.to_latex(f"./OOB_stat_test.txt")


,acc,brier,logloss,ece
calib_methods,,,,
RF_d,,,,
RF_opt,,,,
RF_large,,,,
Platt,\ding{51},\ding{51},\ding{51},\ding{55}
ISO,\ding{51},\ding{51},\ding{51},\ding{51}
Beta,\ding{51},\ding{51},\ding{51},\ding{51}
CRF,\ding{51},\ding{51},,
VA,\ding{51},\ding{51},\ding{51},
tlr,,\ding{51},,


In [6]:

# # Perform an independent samples t-test
# t_stat, p_value = stats.ttest_ind(group1, group2)

# # Print the results
# print("T-Statistic:", t_stat)
# print("P-Value:", p_value)

# # Determine if the difference is statistically significant (using a significance level, e.g., 0.05)
# if p_value < 0.05:
#     print("There is a significant difference between the two groups.")
# else:
#     print("There is no significant difference between the two groups.")